In [1]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import re

In [2]:
DATA_FOLDER = 'data'

In [3]:
dinner_nutrients_df = pd.read_csv('dinner_nutrient.csv')
dinner_ingredients_df = pd.read_csv('dinner_ingredient.csv')

brunch_nutrients_first = pd.read_csv('brunch_nutrient_1-120.csv')
brunch_nutrients_next = pd.read_csv('brunch_nutrient_121-524.csv')
brunch_ingredients_first = pd.read_csv('brunch_ingredient_1-120.csv')
brunch_ingredients_next = pd.read_csv('brunch_ingredient_121-524.csv')

vege_nutrients_df = pd.read_csv('vegetarian_nutrient.csv')
vege_ingredients_df = pd.read_csv('vegetarian_ingredient.csv')

In [4]:
daily_values = pd.read_csv('data/DailyValues', sep=':', header=None, names=['nutrient', 'unit', 'amount'])

## What we need to do

**Cleaning:**
1. Make quantities of nutrients into integers (while respecting units)
2. Remove unnecessary columns
3. Create dataset with URL, recipe name, and ID for all recipes that have nutrient information

**Analysis:**
1. Create nutrient datasets for different types of meals: meat, fish, poultry, and vegetarian
2. Convert macromolecules (carbohydrates, fat, and protein) from grams to caloric values
3. Calculate the caloric percentage of each macromolecule to the total calories of the recipe
4. Compare these percentages with the recommended percentages for a healthy person with a 2000 calorie diet
5. Visualize histograms with quantities of vitamins and minerals for different recipe types to see if there are clear differences
6. We would like to perform a statistical analysis to see how much a difference observed between meal types is due to the different ingredients they contain.

**Visualization of results:**
1. Graphs showing the recommended percentage of macromolecules (for different caloric levels) and the actual percentages that appeared in the recipes for different types of meals
2. Histograms showing differences in nutrient quantities between meal types.

## Lets explore the data and do some cleaning

In [5]:
dinner_nutrients_df.head(20)

,Unnamed: 0,URL,amount,nutrient,recipe
0,0,https://www.allrecipes.com/recipe/222607/smoth...,20.1g,Total Fat,Smothered Chicken Breasts
1,1,https://www.allrecipes.com/recipe/222607/smoth...,8.0g,Saturated Fat,Smothered Chicken Breasts
2,2,https://www.allrecipes.com/recipe/222607/smoth...,124mg,Cholesterol,Smothered Chicken Breasts
3,3,https://www.allrecipes.com/recipe/222607/smoth...,809mg,Sodium,Smothered Chicken Breasts
4,4,https://www.allrecipes.com/recipe/222607/smoth...,377mg,Potassium,Smothered Chicken Breasts
5,5,https://www.allrecipes.com/recipe/222607/smoth...,16.9g,Total Carbohydrates,Smothered Chicken Breasts
6,6,https://www.allrecipes.com/recipe/222607/smoth...,0.5g,Dietary Fiber,Smothered Chicken Breasts
7,7,https://www.allrecipes.com/recipe/222607/smoth...,44g,Protein,Smothered Chicken Breasts
8,8,https://www.allrecipes.com/recipe/222607/smoth...,15g,Sugars,Smothered Chicken Breasts
9,9,https://www.allrecipes.com/recipe/222607/smoth...,29IU,Vitamin A,Smothered Chicken Breasts


We wonder if the amounts for each nutrient always have the same units. Lets verify with Sodium since this is likely to vary a lot. Below only 'head' is displayed, but we checked throughout the dataset.

In [6]:
dinner_nutrients_df = dinner_nutrients_df.drop(["Unnamed: 0"], axis=1)
dinner_ingredients_df = dinner_ingredients_df.drop(["Unnamed: 0"], axis=1)

In [7]:
sodium = dinner_nutrients_df.where(dinner_nutrients_df['nutrient'] == "Sodium")
sodium = sodium.dropna()
sodium = sodium.reset_index(drop=True)
sodium.head()

,URL,amount,nutrient,recipe
0,https://www.allrecipes.com/recipe/222607/smoth...,809mg,Sodium,Smothered Chicken Breasts
1,https://www.allrecipes.com/recipe/15679/asian-...,711mg,Sodium,Asian Beef with Snow Peas
2,https://www.allrecipes.com/recipe/23847/pasta-...,350mg,Sodium,Pasta Pomodoro
3,https://www.allrecipes.com/recipe/50435/fry-br...,2255mg,Sodium,Fry Bread Tacos II
4,https://www.allrecipes.com/recipe/140829/pork-...,356mg,Sodium,Pork Marsala


The unit is the same for each value of a given nutrient, therefore we can drop these letters without worry.

Vitamin A will be completely ignored since it is given in IU. This unit will no longer be valid from 2021. In addition, it cannot be converted into micrograms since this depends on its origin: retinol (pre-formed vitamin A) or beta-carotene (a precursor).

In [8]:
dinner_nutrients_df = dinner_nutrients_df[dinner_nutrients_df.nutrient != "Vitamin A"]

In [9]:
dinner_nutrients_df.head()

,URL,amount,nutrient,recipe
0,https://www.allrecipes.com/recipe/222607/smoth...,20.1g,Total Fat,Smothered Chicken Breasts
1,https://www.allrecipes.com/recipe/222607/smoth...,8.0g,Saturated Fat,Smothered Chicken Breasts
2,https://www.allrecipes.com/recipe/222607/smoth...,124mg,Cholesterol,Smothered Chicken Breasts
3,https://www.allrecipes.com/recipe/222607/smoth...,809mg,Sodium,Smothered Chicken Breasts
4,https://www.allrecipes.com/recipe/222607/smoth...,377mg,Potassium,Smothered Chicken Breasts


In [10]:
#Remove the letters m, c, g, I, U from the string amount and convert it to a float
dinner_nutrients_df.amount = dinner_nutrients_df.amount.apply(lambda x: re.sub('[mcgIU]','', x))
dinner_nutrients_df.amount = dinner_nutrients_df.amount.replace(to_replace='< 1', value='0')
dinner_nutrients_df.amount = dinner_nutrients_df.amount.astype(float)
dinner_nutrients_df.dtypes

URL          object
amount      float64
nutrient     object
recipe       object
dtype: object

Now lets simplify the datasets

In [11]:
#We can identify recipes by their ID
recipe_id = dinner_ingredients_df["URL"].str.extract(r'https://www.allrecipes.com/recipe/(\d+)/.+')
dinner_ingredients_df["recipe_id"] = recipe_id
dinner_ingredients_df = dinner_ingredients_df.drop(["URL"], axis=1)
dinner_ingredients_df.head()

,ingredient,recipe_id
0,"4 (6 ounce) skinless, boneless chicken breast ...",222607
1,1/4 teaspoon salt,222607
2,1/4 teaspoon lemon pepper seasoning,222607
3,1 tablespoon vegetable oil,222607
4,8 strips bacon,222607


In [12]:
print("We have: " + str(len(dinner_ingredients_df["recipe_id"].unique())) + " recipes in the ingredient dataframe.")

We have: 1152 recipes in the ingredient dataframe.


In [13]:
recipe_id = dinner_nutrients_df["URL"].str.extract(r'https://www.allrecipes.com/recipe/(\d+)/.+')
dinner_nutrients_df["recipe_id"] = recipe_id
dinner_nutrients_df.head()

,URL,amount,nutrient,recipe,recipe_id
0,https://www.allrecipes.com/recipe/222607/smoth...,20.1,Total Fat,Smothered Chicken Breasts,222607
1,https://www.allrecipes.com/recipe/222607/smoth...,8.0,Saturated Fat,Smothered Chicken Breasts,222607
2,https://www.allrecipes.com/recipe/222607/smoth...,124.0,Cholesterol,Smothered Chicken Breasts,222607
3,https://www.allrecipes.com/recipe/222607/smoth...,809.0,Sodium,Smothered Chicken Breasts,222607
4,https://www.allrecipes.com/recipe/222607/smoth...,377.0,Potassium,Smothered Chicken Breasts,222607


In [14]:
# let's create a dataframe that contain all the different recipes for dinner with the corresponding name
dinner_recipe_df = dinner_nutrients_df[["recipe_id", "recipe", "URL"]].drop_duplicates()
print("We have in total: " + str(dinner_recipe_df["recipe"].count()) + " different recipes with nutrition info for dinner")
dinner_recipe_df.head()

We have in total: 1128 different recipes with nutrition info for dinner


,recipe_id,recipe,URL
0,222607,Smothered Chicken Breasts,https://www.allrecipes.com/recipe/222607/smoth...
18,15679,Asian Beef with Snow Peas,https://www.allrecipes.com/recipe/15679/asian-...
36,23847,Pasta Pomodoro,https://www.allrecipes.com/recipe/23847/pasta-...
54,50435,Fry Bread Tacos II,https://www.allrecipes.com/recipe/50435/fry-br...
72,140829,Pork Marsala,https://www.allrecipes.com/recipe/140829/pork-...


In [15]:
dinner_nutrients_df = dinner_nutrients_df.drop(["URL", "recipe"], axis=1)
dinner_nutrients_df.head()

,amount,nutrient,recipe_id
0,20.1,Total Fat,222607
1,8.0,Saturated Fat,222607
2,124.0,Cholesterol,222607
3,809.0,Sodium,222607
4,377.0,Potassium,222607


We see that there are recipes that did not give the nutrient values. We ignore these ones by putting exclusively recipe IDs present in the nutrient dataframe into the dataframe linking recipes with URL and name.

**Now we can clean our other datasets easily **

In [16]:
def clean_data(nutrients_df, ingredients_df):
    """
    After exploration, we create a general function to clean all subsequent datasets
    IN: nutrient and ingredient information, in dataframes
    OUT: cleaned nutrient and ingredient dataframes + a new dataframe linking the URL, ID, and name of recipes
    """
    
    nutrients_df = nutrients_df.drop(["Unnamed: 0"], axis=1)
    ingredients_df = ingredients_df.drop(["Unnamed: 0"], axis=1)
    
    #Drop vitamin A
    nutrients_df = nutrients_df[nutrients_df.nutrient != "Vitamin A"]
    
    #Remove the letters m, c, g, I, U from the string amount and convert it to a float
    nutrients_df.amount = nutrients_df.amount.apply(lambda x: re.sub('[mcgIU]','', x))
    nutrients_df.amount = nutrients_df.amount.replace(to_replace='< 1', value='0')
    nutrients_df.amount = nutrients_df.amount.astype(float)
    
    #Identification with ID
    recipe_id = ingredients_df["URL"].str.extract(r'https://www.allrecipes.com/recipe/(\d+)/.+')
    ingredients_df["recipe_id"] = recipe_id
    ingredients_df = ingredients_df.drop(["URL"], axis=1)
    
    recipe_id = nutrients_df["URL"].str.extract(r'https://www.allrecipes.com/recipe/(\d+)/.+')
    nutrients_df["recipe_id"] = recipe_id
    
    #Dataframe linking ID with URL and recipe name
    recipe_df = nutrients_df[["recipe_id", "recipe", "URL"]].drop_duplicates()
    
    nutrients_df = nutrients_df.drop(["URL", "recipe"], axis=1)
    
    return nutrients_df, ingredients_df, recipe_df

In [17]:
brunch_nutrients_df = brunch_nutrients_first.append(brunch_nutrients_next)
brunch_ingredients_df = brunch_ingredients_first.append(brunch_ingredients_next)
brunch_recipes = []

In [18]:
brunch_nutrients_df, brunch_ingredients_df, brunch_recipes = clean_data(brunch_nutrients_df, brunch_ingredients_df)

In [19]:
vege_recipes = []
vege_nutrients_df, vege_ingredients_df, vege_recipes = clean_data(vege_nutrients_df, vege_ingredients_df)

## New insights:

***We will merge all reicpe types together. Then, from this general database we will extract vegetarian, carnivorous, pescatarian, and poultry dishes. We will compare the nutrient profiles of these different types of meals. ***

We remember that the recommended intake for each nutrient is given per day. We cannot specify how many servings of food composes a daily intake, so we cannot compare directly with daily values. Therefore we change our approach. The National Institute of Health gives the suggested percentage of each macromolecule, out of total caloric intake. We will calculate these percentages in our recipes, and compare them with the recommendation. As for vitamins and minerals, we will simply compare their values in the different meal types. Since deficiencies are much more common than overdoses, we will pay particular attention to these.

**Testing making "brunch" data vegetarian:**

If an ingredient contains any word from the document "meats", delete all nutrients with the corresponding ID

In [20]:
f = open('data/meats', 'r')
meat = f.read().splitlines()
f.close()

In [21]:
brunch_copy = brunch_ingredients_df.copy()
brunch_copy = brunch_copy.merge(brunch_recipes, left_on='recipe_id', right_on = 'recipe_id', how='inner', sort=False)

In [22]:
id_to_delete = pd.DataFrame(columns=['URL', 'ingredient', 'recipe', 'recipe_id'])

for i in range(len(meat)):
    id_to_delete = id_to_delete.append(brunch_copy[brunch_copy['ingredient'].str.contains(meat[i])], sort=False)

In [23]:
id_to_delete.head()

,URL,ingredient,recipe,recipe_id
303,https://www.allrecipes.com/recipe/22162/uglies/,1 pound ground beef chuck,Uglies,22162
1708,https://www.allrecipes.com/recipe/33129/corned...,"1 (12 ounce) can corned beef, cut into chunks",Corned Beef Hash,33129
1710,https://www.allrecipes.com/recipe/33129/corned...,1 cup beef broth,Corned Beef Hash,33129
2493,https://www.allrecipes.com/recipe/216981/delux...,"2 pounds cooked corned beef, cubed",Deluxe Corned Beef Hash,216981
3577,https://www.allrecipes.com/recipe/45652/the-sa...,2 pounds ground beef,The Sarge's Goetta - German Breakfast Treat,45652


In [24]:
brunch_nutrient_copy = brunch_nutrients_df.copy()
brunch_nutrient_copy=brunch_nutrient_copy[~brunch_nutrient_copy.isin(id_to_delete['recipe_id'].tolist())]

In [25]:
#Test that the above function worked correctly:

In [26]:
test = brunch_nutrient_copy.where(brunch_nutrient_copy['recipe_id'] == '33129')
test = test.dropna()
test.head(1)

,amount,nutrient,recipe_id


In [27]:
nutrient_copy = brunch_nutrients_df.merge(brunch_recipes, how='inner', left_on='recipe_id', right_on='recipe_id')

In [28]:
val = nutrient_copy.where(nutrient_copy['recipe_id'] == '33129')
val = val.dropna()
val.head(1)

,amount,nutrient,recipe_id,recipe,URL
3366,8.8,Total Fat,33129,Corned Beef Hash,https://www.allrecipes.com/recipe/33129/corned...


**From these insights we will make a generalized function to restrict datasets to different types of meal**

*Below: unused code put in markdown in case we need to verify how to do certain things *

def nutrient_histogram(nutrients_df, name):
    
    """
    Plot a histogram with the amount for a specified nutrient. Show median (black) and FDA recommendation (red).
    IN: nutrient dataframe and name of the nutrient we want to plot
    OUT: histogram with 20 bins for 'amount', median and recommended amounts for the nutrient
    """
    nutrient = nutrients_df.where(nutrients_df['nutrient'] == name)
    nutrient = nutrient.dropna()
    nutrient = nutrient.reset_index(drop=True)
    
    recommended = daily_values.where(daily_values['nutrient'] == name)
    recommended = recommended.dropna()
    recommended = recommended.reset_index(drop=True)
    
    median = nutrient.amount.median()
    rec_value = recommended.at[0, 'amount']

    graph = nutrient.hist(column='amount', bins=50);
    plt.title(n)
    plt.axvline(median, color='k', linestyle='dashed', linewidth=1)
    plt.axvline(rec_value, color='r', linestyle='dashed', linewidth=1)
        
    return median, rec_value, graph

nutrient_list = vege_nutrient_all['nutrient'].unique()
compare_values = pd.DataFrame(columns=['nutrient', 'median', 'rec_value'])

for n in nutrient_list:
    median, rec_value, toto = nutrient_histogram(vege_nutrient_all, n);
    compare_values = compare_values.append({'nutrient': n, 'median':median, 'rec_value': rec_value}, 
                                           ignore_index=True)

## Create vegetarian, meat, poultry, and fish recipe datasets

Just having recipes is not representative of what people are eating since we cannot say how much they eat or how often. So we will instead compare the nutrient profiles of four meal types: vegetarian, meat dishes, fish dishes, and poultry dishes.

In [29]:
def restrict_diet(nutrients_df, ingredients_df, recipes_df, filename, vegetarian=True):
    """
    For all words listed in the file, if that word is contained in ingredients_df, we take the recipe ID.
    If the desired result shall be vegetarian, all recipes with this ID are removed from the list of nutrients.
    If the desired result won't be vegetarian, then only recipes with this ID are kept in the list of nutrients.
    IN: ingredients, nutrients, and recipes, also the name of the file, and a boolean signifying if the result 
    should be vegetarian or not.
    OUT: nutrients from the restricted set of recipes.
    """
    
    f = open(filename, 'r')
    file = f.read().splitlines()
    f.close()
    
    #inner merge ingredients with recipes to associate them with recipe IDs that have nutrients
    ingredients_copy = ingredients_df.copy()
    ingredients_copy = ingredients_copy.merge(recipes_df, left_on='recipe_id', right_on='recipe_id', 
                                              how='inner', sort=False)
    
    #Create a dataframe of IDs, corresponding to recipes that contain words in the given file
    ids = pd.DataFrame(columns=['URL', 'ingredient', 'recipe', 'recipe_id'])
    for i in range(len(file)):
        ids = ids.append(ingredients_copy[ingredients_copy['ingredient'].str
                                                            .contains(file[i])], sort=False)
    
    nutrients_copy = nutrients_df.copy()
    
    if vegetarian:
        #The vegetarian recipes are those not listed in ids; we keep their nutrients
        nutrients_copy=nutrients_copy[~nutrients_copy.isin(ids['recipe_id'].tolist())]
    else:
        nutrients_copy = nutrients_copy[nutrients_copy.isin(ids['recipe_id'].tolist())]
    
    return nutrients_copy

In [30]:
#creation of large dataset containing all recipes we have

allrecipes_nutrients = pd.DataFrame(columns=['amount', 'nutrient', 'recipe_id'])
allrecipes_nutrients = allrecipes_nutrients.append(dinner_nutrients_df, ignore_index = True, sort = False)
allrecipes_nutrients = allrecipes_nutrients.append(brunch_nutrients_df, ignore_index = True, sort = False)
allrecipes_nutrients = allrecipes_nutrients.append(vege_nutrients_df, ignore_index = True, sort = False)

allrecipes_ingredients = pd.DataFrame(columns=['ingredient', 'recipe_id'])
allrecipes_ingredients = allrecipes_ingredients.append(dinner_ingredients_df, ignore_index = True, sort = False)
allrecipes_ingredients = allrecipes_ingredients.append(brunch_ingredients_df, ignore_index = True, sort = False)
allrecipes_ingredients = allrecipes_ingredients.append(vege_ingredients_df, ignore_index = True, sort = False)

allrecipes = pd.DataFrame(columns=['recipe', 'URL'])
allrecipes = allrecipes.append(dinner_recipe_df, ignore_index = True, sort = False)
allrecipes = allrecipes.append(brunch_recipes, ignore_index = True, sort = False)
allrecipes = allrecipes.append(vege_recipes, ignore_index = True, sort = False)

In [31]:
allrecipes.shape

(3110, 3)

In [32]:
#vegetarian meals
#the words from each file must be removed separately
temp_vege_nutrients = restrict_diet(allrecipes_nutrients, allrecipes_ingredients, allrecipes, 
                                          filename = 'data/meats')
temp_vege_nutrients = restrict_diet(temp_vege_nutrients, allrecipes_ingredients, allrecipes, 
                                     filename = 'data/fish')
vege_nutrients = restrict_diet(temp_vege_nutrients, allrecipes_ingredients, allrecipes, 
                                     filename = 'data/poultry')

In [33]:
#meat, fish, and poultry meals
meat_nutrients = restrict_diet(allrecipes_nutrients, allrecipes_ingredients, allrecipes, 
                                     filename = 'data/meats', vegetarian = False)
fish_nutrients = restrict_diet(allrecipes_nutrients, allrecipes_ingredients, allrecipes, 
                                     filename = 'data/fish', vegetarian = False)
poultry_nutrients = restrict_diet(allrecipes_nutrients, allrecipes_ingredients, allrecipes, 
                                     filename = 'data/poultry', vegetarian = False)

# Convert macromolecules to calories
We want to calculate macromolecules as % total calories, as we have recommended percentages as Dietary Reference Intakes, which are acceptable macronutrient distribution ranges from this [page](https://www.ncbi.nlm.nih.gov/books/NBK56068/table/summarytables.t5/?report=objectonly).

We can convert total fat grams to kilocalories by the following calculation: (Total Fat Weight in grams * 9)

We can convert carbohydrates: (Saturated Fat in grams * 4)

We can convert Protein: (Protein in grams * 4)

Only these three macromolecules contribute in any significant way to the total calories of a recipe, so the total calories can be calculated by adding them. Knowing the total calories of the recipe, it is easy to calculate what percentage of this total is due to each of the three macromolecules above: 100 * calories of macromolecule / total calories.

In [77]:
def calories(nutrients_df):
    """
    Convert amounts (in grams) for Fat, Carbohydrates, and Protein into kilocalories
    IN: dataframe with nutrients and corresponding amounts
    OUT: dataframe in the same format but where amounts are in kcal
    """
    fat_df = nutrients_df.where(nutrients_df.nutrient == 'Total Fat')
    fat_df = fat_df.dropna()
    fat_df['amount'] = fat_df['amount'] * 9
    fat_df = fat_df.set_index(fat_df['recipe_id'], drop=False)
    
    carb_df = nutrients_df.where(nutrients_df.nutrient == 'Total Carbohydrates')
    carb_df = carb_df.dropna()
    carb_df['amount'] = carb_df['amount'] * 4
    carb_df = carb_df.set_index(carb_df['recipe_id'], drop=False)
    
    prot_df = nutrients_df.where(nutrients_df.nutrient == 'Protein')
    prot_df = prot_df.dropna()
    prot_df['amount'] = prot_df['amount'] * 4
    prot_df = prot_df.set_index(prot_df['recipe_id'], drop=False)
    
    cal_nutrient_df = pd.DataFrame(columns=['amount', 'nutrient', 'recipe_id'])
    cal_nutrient_df = cal_nutrient_df.append(fat_df, ignore_index=False, sort=True)
    cal_nutrient_df = cal_nutrient_df.append(carb_df, ignore_index=False, sort=True)
    cal_nutrient_df = cal_nutrient_df.append(prot_df, ignore_index=False, sort=True)
    
    cal_nutrient_df.sort_index(axis=0, inplace=True, ascending=False)
    
    cal_nutrient_df = cal_nutrient_df.rename(columns={'amount': 'amount_kcal'})
    
    return cal_nutrient_df

In [81]:
vege_kcal_nutrients = calories(vege_nutrients)
vege_kcal_nutrients.head(6)

,amount_kcal,nutrient,recipe_id
99211,9.0,Total Fat,99211
99211,6.8,Protein,99211
99211,94.0,Total Carbohydrates,99211
98539,331.6,Total Carbohydrates,98539
98539,400.5,Total Fat,98539
98539,209.6,Protein,98539


In [82]:
meat_kcal_nutrients = calories(meat_nutrients)
fish_kcal_nutrients = calories(fish_nutrients)
poultry_kcal_nutrients = calories(poultry_nutrients)

### Plotting nutrient profile histograms

Ideally, we will be able to plot on the same histogram data from all meal types

In [34]:
def nutrient_histogram(nutrients_df, name):
    
    """
    Plot a histogram with the amount for a specified nutrient. Show median.
    IN: nutrient dataframe and name of the nutrient we want to plot
    OUT: histogram with 20 bins for 'amount', median and recommended amounts for the nutrient
    """
    nutrient = nutrients_df.where(nutrients_df['nutrient'] == name)
    nutrient = nutrient.dropna()
    nutrient = nutrient.reset_index(drop=True)
    
    median = nutrient.amount.median()

    graph = nutrient.hist(column='amount', bins=50);
    plt.title(n)
    plt.axvline(median, color='k', linestyle='dashed', linewidth=1)
        
    return median, graph